In [1]:
import requests
import os
import pandas as pd
from data_collection import fetch_and_save_river_data
import datetime

In [3]:
cornwall_stations = pd.read_csv('river_station_info_cornwall.csv')

In [4]:
station_ids = cornwall_stations.id.to_list()
station_ids

[380,
 382,
 384,
 386,
 387,
 391,
 393,
 394,
 395,
 397,
 398,
 399,
 400,
 1349,
 1351,
 1352,
 8256,
 8340,
 16446,
 17891,
 355,
 357,
 358,
 361,
 362,
 364,
 365,
 370,
 375,
 1335,
 1339,
 1340,
 1341,
 1342,
 8227,
 14453,
 14454]

In [5]:
# Current date
now = datetime.datetime.now()

# Yesterday's date
yesterday = now - datetime.timedelta(days=1)

# Start date as January 1st, 2000
start_date = datetime.datetime(2000, 1, 1)

# Format the dates to the ISO 8601 format (as strings)
start_date_str = start_date.strftime('%Y-%m-%dT%H')
yesterday_str = yesterday.strftime('%Y-%m-%dT%H')

start_date


datetime.datetime(2000, 1, 1, 0, 0)

In [7]:
fetch_and_save_river_data(station_ids=[355], start_date=start_date, end_date=yesterday)
from transform_data import extract_time_values_from_csv
cleaned = extract_time_values_from_csv('/Users/antonfreidin/water_project/get_river_data/data/station_355.csv')

Data for station 355 saved successfully in get_river_data/data/station_355.csv.


In [9]:
cleaned.to_csv('gpt_355.csv')

In [5]:
fetch_and_save_river_data(station_ids=station_ids, start_date=start_date, end_date=yesterday)

Data for station 380 saved successfully in get_river_data/data/station_380.csv.
Data for station 382 saved successfully in get_river_data/data/station_382.csv.
Data for station 384 saved successfully in get_river_data/data/station_384.csv.
Request failed for station 386, status code: 500
Data for station 387 saved successfully in get_river_data/data/station_387.csv.
Request failed for station 391, status code: 500
Data for station 393 saved successfully in get_river_data/data/station_393.csv.
Data for station 394 saved successfully in get_river_data/data/station_394.csv.
Data for station 395 saved successfully in get_river_data/data/station_395.csv.
Data for station 397 saved successfully in get_river_data/data/station_397.csv.
Data for station 398 saved successfully in get_river_data/data/station_398.csv.
Request failed for station 399, status code: 500
Data for station 400 saved successfully in get_river_data/data/station_400.csv.
Request failed for station 1349, status code: 500
Dat

In [6]:
failed = [386,391,399,1349,16446,358,375]

In [7]:
fetch_and_save_river_data(station_ids=failed, start_date=start_date, end_date=yesterday)

Request failed for station 386, status code: 500
Request failed for station 391, status code: 500
Request failed for station 399, status code: 500
Request failed for station 1349, status code: 500
Request failed for station 16446, status code: 500
Request failed for station 358, status code: 500
Request failed for station 375, status code: 500


# EDA 


In [1]:
import os
import pandas as pd
from ast import literal_eval

In [2]:
df = pd.read_csv('get_river_data/data/station_8256.csv', usecols=['values'])

In [3]:
import ast

# Safety check: ensure the column is not aggregated into one entry
print(df.shape)  # Should show more than 1 row if data is spread across multiple rows

# Convert string representation of lists into actual lists of dictionaries
df['values'] = df['values'].apply(ast.literal_eval)

# Exploding the list into rows
df = df.explode('values')

# Display the result
print(df.head())


(1, 1)
                                            values
0  {'time': '2014-03-04T04:15:00', 'value': 0.249}
0  {'time': '2014-03-04T04:30:00', 'value': 0.244}
0  {'time': '2014-03-04T04:45:00', 'value': 0.245}
0  {'time': '2014-03-04T05:00:00', 'value': 0.241}
0  {'time': '2014-03-04T05:15:00', 'value': 0.245}


In [4]:
import pandas as pd

# Assuming df is your DataFrame after the literal_eval step
# If the DataFrame has not been exploded properly (which it should as per your data sample):
if not isinstance(df.at[0, 'values'], list):
    df['values'] = df['values'].apply(lambda x: [x])  # Ensure it's a list
df = df.explode('values')  # Now explode the DataFrame

# Convert dictionaries to separate columns
df = pd.concat([df.drop('values', axis=1), df['values'].apply(pd.Series)], axis=1)

# Convert 'time' to datetime format
df['time'] = pd.to_datetime(df['time'])

# Display the DataFrame structure
print(df.head())
print(df.info())


                 time  value
0 2014-03-04 04:15:00  0.249
0 2014-03-04 04:30:00  0.244
0 2014-03-04 04:45:00  0.245
0 2014-03-04 05:00:00  0.241
0 2014-03-04 05:15:00  0.245
<class 'pandas.core.frame.DataFrame'>
Index: 227880 entries, 0 to 0
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   time    227880 non-null  datetime64[ns]
 1   value   227880 non-null  float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 5.2 MB
None


In [12]:
df.set_index('time')

,value
time,
2014-03-04 04:15:00,0.249
2014-03-04 04:30:00,0.244
2014-03-04 04:45:00,0.245
2014-03-04 05:00:00,0.241
2014-03-04 05:15:00,0.245
...,...
2023-11-23 03:00:00,0.551
2023-11-23 03:15:00,0.559
2023-11-23 03:30:00,0.552
